In [3]:
import tensorflow as tf
import numpy as np
tf.__version__


'1.13.1'

In [6]:
!cat /proc/cpuinfo

'cat'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [7]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


## 텐서플로를 이용한 프로그램 작성 시,
### 1. 상수, 변수, 텐서연산 등의 노드와 엣지를 먼저 정의하고,
### 2. 세션을 만들고 그 세션을 통해 노드간의 데이터(텐서) 연산 수행

## 상수 노드 - tf.constant()
상수 노드 정의 , 한번정하면 바꿀수가 없다.

In [20]:
import tensorflow as tf

#상수값을 저장하는 노드를 만들기 위해서 상수 노드 tf.constant 정의
a = tf.constant(1.0, name = 'a')
b = tf.constant(2.0, name = 'b')
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
#세션을 만들지 않고 print 와 같은 명령문을 실행하면, 저장된 값이 아닌
#현재 정의 되어 있는 노드의 상태(노드타입, shape 등) 출력됨
print(a)
print(a+b)
print(c)
#노드간의 연산을 위해 세션 생성
sess = tf.Session()
#세션을 통해(sess.run()) 노드에 값이 할당되고 노드간의 텐서를 흘려보내면서
#tensor flow 연산과 명령문 등이 실행됨
print("a, b = ", sess.run([a,b]))
print("c = ",sess.run(c))
print("a + b = ",sess.run([a+b]))
print("c + 1",sess.run(c+1.0))

sess.close()

Tensor("a_4:0", shape=(), dtype=float32)
Tensor("add_13:0", shape=(), dtype=float32)
Tensor("Const_4:0", shape=(2, 2), dtype=float32)
a, b =  [1.0, 2.0]
c =  [[1. 2.]
 [3. 4.]]
a + b =  [3.0]
c + 1 [[2. 3.]
 [4. 5.]]


## 변수노드 - tf.Varialbe()
가중치나 바이어스처럼 계속 업데이트되는 변수는 텐서플로에서 변수 노드로 정의

In [19]:
import tensorflow as tf
# random_uniform 특정 구간에서 랜덤값을 리턴해준다.
W1 = tf.Variable(tf.random_normal([1]))  #가우시안 분포를 리턴해준다.
b1 = tf.Variable(tf.random_normal([1]))  # b1 = np.random.rand(1) 과 비슷함

W2 = tf.Variable(tf.random_normal([1,2])) 
b2 = tf.Variable(tf.random_normal([1,2]))
#세션 생성
sess = tf.Session()
#변수노드 값 초기화를 위해서 반드시 실행
sess.run(tf.global_variables_initializer())

for step in range(3):
    W1 = W1 - step
    b1 = b1 - step
    
    W2 = W2 - step
    b2 = b2 - step
    
    print("step = ", step, ", W1 = ", sess.run(W1), ", b1 = ", sess.run(b1))
    print("step = ", step, ", W2 = ", sess.run(W2), ", b2 = ", sess.run(b2))

sess.close()

step =  0 , W1 =  [0.77462286] , b1 =  [-0.9277637]
step =  0 , W2 =  [[0.64589316 1.508661  ]] , b2 =  [[0.1469841 0.7898316]]
step =  1 , W1 =  [-0.22537714] , b1 =  [-1.9277637]
step =  1 , W2 =  [[-0.35410684  0.50866103]] , b2 =  [[-0.8530159  -0.21016842]]
step =  2 , W1 =  [-2.225377] , b1 =  [-3.9277637]
step =  2 , W2 =  [[-2.354107 -1.491339]] , b2 =  [[-2.853016  -2.2101684]]


## 플레이스홀더 노드 - tf.placeholder
플레이스 홀더 노드는 머신러닝/딥러닝에서 입력데이터(input), 정답데이터(targer)를 넣어주기 위한 용도로 주로 사용됨

In [18]:
import tensorflow as tf

#플레이스홀더 노드 정의
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a + b

sess = tf.Session()
#플레이스홀더 노드에 실제 값을 넣어줄때는 sess.run 첫번째 인자로 실행하고자 하는 연산을 넣어주고,
#두번째 인자에는 실제로 넣을 값들을 Dictionary형태로 넣어주는 feed_dict을 선언하고,feed_dict부분에
#플레이스홀더에 넣을 값을 지정해줌.
print(sess.run(c, feed_dict = {a: 1.0, b: 3.0}))
print(sess.run(c, feed_dict = {a: [1.0, 2.0], b: [3.0, 4.0]}))

d = 100 * c

print(sess.run(d, feed_dict={a: 1.0, b: 3.0}))
print(sess.run(d, feed_dict={a: [1.0,2.0], b: [3.0,4.0]}))

sess.close()

4.0
[4. 6.]
400.0
[400. 600.]


## Linear Regression (multi-variable example)

In [22]:
import tensorflow as tf
import numpy as np

loaded_data = np.loadtxt('./data-01.csv', delimiter = ',')

x_data = loaded_data[ :, 0:-1]
t_data = loaded_data[ :, [-1]]

print("x_data.shape = ", x_data.shape)
print("t_data.shape = ", t_data.shape)

x_data.shape =  (25, 3)
t_data.shape =  (25, 1)


### 가중치 노드 W, 바이어스 노드 b를 정의하고 feed_dict을 통해 데이터를 넣어주기 위해서 입력데이터 노드와 정답데이터 노드를 정의함.

In [23]:
W = tf.Variable(tf.random_normal([3,1]))
b = tf.Variable(tf.random_normal([1]))

X = tf.placeholder(tf.float32, [None,3])
T = tf.placeholder(tf.float32, [None,1])

### 현재의 X,W,b를 바탕으로 선형회귀 값 y를 계산하고, y와 정답 T를 이용하여 손실함수 정의

In [24]:
y = tf.matmul(X, W) + b
loss = tf.reduce_mean(tf.square(y - T))

### 가중치 W, 바이어스 b를 최적화 하기 위해서 경사하강법(Gradient Descent Algorithm)을 적용한 optimizer 정의함 이처럼 TensorFlow는 다양한 optimizer를 적용하여 손실함수를 최소화 하고, 최종적으로 W,b를 최적화 시킴.

In [25]:
learning_rate = 1e-5
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

### 노드 / 연산 실행

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(8001):
        # _ 하나만 사용하면 쓸모없는 데이타로 정의한다.
        loss_val, y_val, _ = sess.run([loss, y, train], feed_dict = {X: x_data, T: t_data})  
        
        if step % 400 == 0:
            print("step = ", step, ", loss_val = ", loss_val)
    
    print("\nprediction is ", sess.run(y, feed_dict= {X: [[100, 98, 81] ]}))

step =  0 , loss_val =  69650.984
step =  400 , loss_val =  69.80117
step =  800 , loss_val =  52.150166
step =  1200 , loss_val =  39.528126
step =  1600 , loss_val =  30.475906
step =  2000 , loss_val =  23.962696
step =  2400 , loss_val =  19.25942
step =  2800 , loss_val =  15.849609
step =  3200 , loss_val =  13.36681
step =  3600 , loss_val =  11.5505295
step =  4000 , loss_val =  10.215109
step =  4400 , loss_val =  9.228001
step =  4800 , loss_val =  8.494263
step =  5200 , loss_val =  7.945661
step =  5600 , loss_val =  7.533046
step =  6000 , loss_val =  7.220804
step =  6400 , loss_val =  6.983103
step =  6800 , loss_val =  6.801017
step =  7200 , loss_val =  6.6607203
step =  7600 , loss_val =  6.55201
step =  8000 , loss_val =  6.4673157

prediction is  [[180.93687]]
